# 문장 추천 인터페이스
- input : str 형태. 예) 'I am a boy.'
- output : list of str 형태. 예) ['I am a girl.', 'You are a boy.']

## 사용법
아래에 있는 코드를 복사하여 sent_interface.py를 만든다.

from sent_interface import similar_sents

similar_sents('I am a boy.')

In [1]:
import pickle
import re
from nltk import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

C:\Users\Bangjungyip\AppData\Local\conda\conda\envs\gensim\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
with open('corpus.pickle', 'rb') as f:
    original_sents = pickle.load(f)

with open('common_texts.pickle', 'rb') as f:
    common_texts = pickle.load(f)

model = Doc2Vec.load('doc2vec.model')

In [5]:
def similar_sents(input_sent):
    p = re.compile('[a-zA-Z]+')  # 알파벳으로만 된 단어 패턴

    input_token = word_tokenize(input_sent.lower())
    
    average_n = 10
    vector = 0
    for k in range(average_n):
        vector += model.infer_vector(input_token)
    vector /= average_n

    # Search for the most similar sentences
    similar_sentences = model.docvecs.most_similar(positive=[vector], topn=10)
    
    # topn개의 문장 중에서 이상한 문장 지우기
    good_sents = []
    
    for sent_index in range(len(similar_sentences)):
        sent = original_sents[similar_sentences[sent_index][0]]
        
        is_good_sent = True
        
        # 문장의 단어 수가 5보다 작으면 좋은 문장이 아니다.
        if len(sent) < 5:
            is_good_sent = False
            
        # 한 단어의 길이가 20을 넘지 않고, 영어가 아닌 문장이 너무 많이 들어가 있지 않아야
        good_word_cnt = 0
        for word in sent:
            if len(word) >= 20:
                is_good_sent = False
                break
            if p.match(word):
                good_word_cnt += 1
                
        # 특수 문자가 너무 많을 경우
        if good_word_cnt < 0.67 * len(sent):
            is_good_sent = False
            
        # 이미 있는 문장의 경우
        if sent in good_sents:
            is_good_sent = False
            
        if is_good_sent:
            good_sents.append(sent)
    
    num_sent = len(good_sents) if len(good_sents) < 3 else 3
    
    return good_sents[:num_sent]

In [17]:
similar_sents('Harvard Business Review Polska')

['Il mio amico fraterno, Luigi, sempre pronto con idee di business mai realizzate.',
 'Chapter 4 is a review of concepts around business   3 | P a g e',
 'com / difference between proactive reactive business strategy 62157.html.']